In [ ]:
pip install concepts

In [ ]:
import concepts
from concepts import Context
import numpy as np

In [ ]:
# Sample data: Books and their genres
books = ['The Hobbit', 'Dune', 'The Martian', '1984', 'Pride and Prejudice',
         'The Da Vinci Code', 'Neuromancer', 'The Hunger Games', 'Gone Girl',
         'Harry Potter', 'Brave New World', 'The Fault in Our Stars',
         'The Girl with the Dragon Tattoo', 'A Game of Thrones', 'Ender’s Game',
         'Divergent', 'Me Before You', 'Sherlock Holmes: A Study in Scarlet',
         'Frankenstein', 'Red Queen','Demo']
genres = ['Fantasy', 'Sci-Fi', 'Mystery', 'Dystopian', 'Romance', 'Thriller']

* The Hobbit
* Dune
* The Martian
* 1984
* Pride and Prejudice
* The Da Vinci Code
* Neuromancer
* The Hunger Games
* Gone Girl
* Harry Potter
* Brave New World
* The Fault in Our Stars
* The Girl with the Dragon Tattoo
* A Game of Thrones
* Ender’s Game
* Divergent
* Me Before You
* Sherlock Holmes: A Study in Scarlet
* Frankenstein
* Red Queen

In [ ]:
# Incidence matrix: rows correspond to books, columns to genres (1 if book has genre)
incidence_matrix = [
    [1, 0, 0, 0, 0, 0],  # The Hobbit: (Fantasy)
    [1, 1, 0, 0, 0, 0],  # Dune: (Fantasy, Sci-Fi)
    [0, 1, 0, 0, 0, 0],  # The Martian: (Sci-Fi)
    [0, 1, 0, 1, 0, 0],  # 1984: (Sci-Fi, Dystopian)
    [0, 0, 0, 0, 1, 0],  # Pride and Prejudice: (Romance)
    [0, 0, 1, 0, 0, 1],  # The Da Vinci Code: (Mystery, Thriller)
    [0, 1, 0, 1, 0, 1],  # Neuromancer: (Sci-Fi, Dystopian, Thriller)
    [1, 0, 0, 1, 0, 0],  # The Hunger Games: (Fantasy, Dystopian)
    [0, 0, 1, 0, 0, 1],  # Gone Girl: (Mystery, Thriller)
    [1, 0, 0, 0, 0, 0],  # Harry Potter: (Fantasy)
    [0, 1, 0, 1, 0, 0],  # Brave new world: (Sci-Fi, Dystopian)
    [0,	0, 0,	0, 1,	0],  # The Fault in our Stars: (Romance)
    [0,	0, 1,	0, 0,	1],  # The Girl with the Dragon Tattoo: (Mystry, Thriller)
    [1,	0, 0,	0, 1,	0],  # The Game of Thrones: (Fantasy, Romance)
    [0,	1, 0,	0, 0,	0],  # Ender's Game: (Sci-Fi)
    [0,	1, 0,	1, 0,	0],  # Divergent: (Sci-Fi, Dystopian)
    [0,	0, 0,	0, 1,	0],  # Me Before you: (Romance)
    [0,	0, 1,	0, 0, 0],  # Sherlock Holmes: A Study in Scarlet: (Mystry)
    [0,	1, 1,	0, 0,	0],  # Frankenstein: (Sci-Fi, Mystry)
    [1,	0, 0,	1, 1,	0],  # Red Queen: (Fantasy, Dystopian, Romance)
    [0, 0, 0, 0, 0, 0],  # Demo: (No taste)
]

In [ ]:
incidence_matrix = np.array(incidence_matrix)
incidence_matrix.shape

(21, 6)

In [ ]:
# Convert matrix to boolean values for formal contexting
bools = [[bool(bit) for bit in row] for row in incidence_matrix]

In [ ]:
# Step 1: Create formal context (object=books, attributes=genres)
context = Context(books, genres, bools)

In [ ]:
# Step 2: Generate concept lattice from formal context
lattice = context.lattice
lattice

<Lattice object of 5 atoms 16 concepts 6 coatoms at 0x7bf9e0dd1ad0>

In [ ]:
# Step 3: Map each book to its associated concepts in the lattice
book_concepts = {book: [] for book in books}
for concept in lattice:
    for book in concept.extent:
        book_concepts[book].append(concept)

In [ ]:
def get_book_index(book_name, book_list):
    try:
        return book_list.index(book_name)
    except ValueError:
        return -1

In [ ]:
def recommend(target_book, books, bool_matrix):
    """Recommend books based on direct genre overlap (shared 1s in matrix)."""
    idx = get_book_index(target_book, books)
    if idx == -1:
        return []

    target_vector = bool_matrix[idx]
    similarities = {}

    for i, other_vector in enumerate(bool_matrix):
        if i == idx:
            continue

        shared_genres = sum(a and b for a, b in zip(target_vector, other_vector))
        if shared_genres > 0:
            similarities[books[i]] = shared_genres

    sorted_books = sorted(similarities.items(), key=lambda x: (-x[1], x[0]))
    return sorted_books[:5]#top 15 books will be recommended

In [ ]:
def match_book(user_input, book_list):
    user_input = user_input.lower().strip()
    lower_map = {book.lower(): book for book in book_list}

    if user_input in lower_map:
        return lower_map[user_input]

    matches = [book for book in book_list if user_input in book.lower()]

    if len(matches) == 1:
        return matches[0]
    elif len(matches) > 1:
        print("Multiple matches found:")
        for i, book in enumerate(matches, 1):
            print(f"{i}. {book}")
        choice = input("Enter the number of the book you meant: ").strip()
        if choice.isdigit() and 1 <= int(choice) <= len(matches):
            return matches[int(choice) - 1]
        else:
            return None
    else:
        return None

In [ ]:
user_input = input("Enter a book name: ")
target = match_book(user_input, books)

if target is None:
    print("Book not found or unclear. Please try again with a clearer name.")
else:
    recommendations = recommend(target, books, bools)

    if not recommendations:
        print(f"No recommendations found for '{target}' based on shared genres.")
    else:
        print(f"\nTop recommendations for '{target}':\n")
        for i, (book, score) in enumerate(recommendations, 1):
            print(f"{i}. {book} (Shared genres: {score})")

Enter a book name: dune

Top recommendations for 'Dune':

1. 1984 (Shared genres: 1)
2. A Game of Thrones (Shared genres: 1)
3. Brave New World (Shared genres: 1)
4. Divergent (Shared genres: 1)
5. Ender’s Game (Shared genres: 1)
